<a href="https://colab.research.google.com/github/Priesemann-Group/covid19_inference_forecast/blob/cleaning_code/scripts/example_script_covid19_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/Priesemann-Group/covid19_inference_forecast.git@cleaning_code

  Cloning https://github.com/Priesemann-Group/covid19_inference_forecast.git (to revision cleaning_code) to /tmp/pip-req-build-0yysoruw
  Running command git clone -q https://github.com/Priesemann-Group/covid19_inference_forecast.git /tmp/pip-req-build-0yysoruw
  Running command git checkout -b cleaning_code --track origin/cleaning_code
  Switched to a new branch 'cleaning_code'
  Branch 'cleaning_code' set up to track remote branch 'cleaning_code' from 'origin'.
  Created wheel for covid19-inference: filename=covid19_inference-0.0.0-cp36-none-any.whl size=5427 sha256=1740c50eb69d1f72a2ec48b87d8406c9494012bdc1f0ecf5b53b5d2fb1278637
  Stored in directory: /tmp/pip-ephem-wheel-cache-ckeruzsd/wheels/0b/b2/1d/7bff72ae35855db34fab454bb6b6cbce5566675e123ec2f41e
Successfully built covid19-inference


In [2]:
import sys
import datetime

import numpy as np
import pymc3 as pm

import covid19_inference as cov19

confirmed_cases = cov19.get_jhu_confirmed_cases()

date_data_begin = datetime.datetime(2020,3,1)
date_data_end   = cov19.get_last_date(confirmed_cases)
#date_data_end   = datetime.datetime(2020,3,28)
num_days_data = (date_data_end-date_data_begin).days
diff_data_sim = 16 # should be significantly larger than the expected delay, in 
                   # order to always fit the same number of data points.
num_days_future = 28
date_begin_sim = date_data_begin - datetime.timedelta(days = diff_data_sim)
date_end_sim   = date_data_end   + datetime.timedelta(days = num_days_future)
num_days_sim = (date_end_sim-date_begin_sim).days


cases_obs = cov19.filter_one_country(confirmed_cases, 'Germany',
                                     date_data_begin, date_data_end)

change_points = [dict(prior_mean_date_begin_transient = datetime.datetime(2020,3,9),
                      prior_median_λ = 0.2),
                 dict(prior_mean_date_begin_transient = datetime.datetime(2020,3,16),
                      prior_sigma_date_begin_transient = 1,
                      prior_median_λ = 1/8,
                      prior_sigma_λ = 0.2),
                 dict(prior_mean_date_begin_transient = datetime.datetime(2020,3,23),
                      prior_sigma_date_begin_transient = 1,
                      prior_median_λ = 1/8/2,
                      prior_sigma_λ = 0.2)]

model = cov19.SIR_model_with_change_points(np.diff(cases_obs),
                                                       change_points,
                                                       date_begin_sim,
                                                       num_days_sim,
                                                       diff_data_sim)

trace = pm.sample(model=model, init='advi+adapt_diag')


Hi


Auto-assigning NUTS sampler...
Initializing NUTS using advi...
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
Average Loss = 306.55:   6%|▋         | 12992/200000 [00:41<10:03, 309.93it/s]
Convergence achieved at 13000
Interrupted at 12,999 [6%]: Average Loss = 392.69
Sequential sampling (2 chains in 1 job)
NUTS: [σ_obs, delay, μ, transient_len_2, transient_len_1, transient_len_0, transient_begin_2, transient_begin_1, transient_begin_0, λ_3, λ_2, λ_1, λ_0, I_begin]
100%|██████████| 1000/1000 [04:19<00:00,  3.85it/s]
The estimated number of effective samples is smaller than 200 for some parameters.


<MultiTrace: 2 chains, 500 iterations, 27 variables>